In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext

In [2]:
df_silverBreweries = (
    spark
    .read
    .option("basePath", f"../datalake/silver/breweries/")
    .parquet(f"../datalake/silver/breweries/*")
)

In [3]:
df_aggBreweries = (
    df_silverBreweries
    .groupBy(
        F.col('brewery_type'),
        F.col('country'),
        F.col('state')
    )
    .count()
)

In [4]:
df_aggBreweries.show()

+------------+-------------+---------------+-----+
|brewery_type|      country|          state|count|
+------------+-------------+---------------+-----+
|    regional|United States|         Kansas|    1|
|       large|      Austria|     Steiermark|    4|
|       large|United States|     California|   15|
|       micro|      Ireland|      Tipperary|    2|
|     brewpub|  South Korea|      Gangwondo|    7|
|     brewpub|United States|     California|  253|
|       micro|United States|       Maryland|   52|
|        nano|United States|   North Dakota|    2|
|        nano|United States|  West Virginia|    1|
|         bar|United States|         Nevada|    1|
|       large|United States|        Georgia|    3|
|        nano|United States|      Tennessee|    1|
|       micro|      Ireland|         Offaly|    2|
|     brewpub|  South Korea|    Jeollabukdo|    3|
|     brewpub|  South Korea|          Deagu|    1|
|       micro|United States|   Rhode Island|   20|
|    planning|United States|   

In [5]:
(
    df_aggBreweries
    .coalesce(1)
    .write    
    .mode('overwrite')
    .parquet(f"../datalake/gold/aggBreweries/")
)